# simple script to allow the visualization of insulation QTL as a bedgraph

In [1]:
import pandas as pd
import numpy as np
import hicstraw 
from multiprocessing import Pool
from functools import partial
import glob
import os
import plotly.express as px
import math
import matplotlib.pyplot as plt
from matplotlib import colors
from pandarallel import pandarallel
import cooler
import cooltools
import pybedtools as pbed
pandarallel.initialize()
from scipy import stats, special
from statsmodels.stats import multitest
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.io as pio
import seaborn as sns

from functools import reduce

os.makedirs("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/", exist_ok = True)
pbed.helpers.set_tempdir("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/")
bed_genome_file = "/mnt/iusers01/jw01/mdefscs4/hg38.genome"

plt.rcParams['svg.fonttype'] = 'none'

base_dir = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis"

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
ins_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/ins_nominal_CD8_merged.txt", sep = " ")
ins_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/ins_nominal_CD4_merged.txt", sep = " ")

In [ ]:
loop_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/loop_nominal_CD8_merged.txt", sep = " ")
loop_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/loop_nominal_CD4_merged.txt", sep = " ")

In [ ]:
ins_nominal_CD8["log_pval"] = -np.log10(ins_nominal_CD8["nom_pval"])
ins_nominal_CD4["log_pval"] = -np.log10(ins_nominal_CD4["nom_pval"])
loop_nominal_CD8["log_pval"] = -np.log10(loop_nominal_CD8["nom_pval"])
loop_nominal_CD4["log_pval"] = -np.log10(loop_nominal_CD4["nom_pval"])

In [ ]:
loop_nominal_CD8["phe_from_end"] = loop_nominal_CD8["phe_from"] + 5000
loop_nominal_CD4["phe_from_end"] = loop_nominal_CD4["phe_from"] + 5000
loop_nominal_CD8["phe_to_start"] = loop_nominal_CD8["phe_to"] - 5000
loop_nominal_CD4["phe_to_start"] = loop_nominal_CD4["phe_to"] - 5000

In [ ]:
for wanted_snp in ["rs11209051","rs2631367","rs13396472","rs12936231","rs13140464","rs2353679"]:
    ins_nominal_CD8[ins_nominal_CD8["var_id"] == wanted_snp][["phe_chr","phe_from","phe_to","slope","log_pval"]].to_csv(
        f"{base_dir}/QTL_analysis/ins_output/ins_CD8_{wanted_snp}.bdg", sep = "\t", index = False)
    ins_nominal_CD4[ins_nominal_CD4["var_id"] == wanted_snp][["phe_chr","phe_from","phe_to","slope","log_pval"]].to_csv(
        f"{base_dir}/QTL_analysis/ins_output/ins_CD4_{wanted_snp}.bdg", sep = "\t", index = False)
    
    df = loop_nominal_CD8[loop_nominal_CD8["var_id"] == wanted_snp].copy()
    df['combined'] = df['phe_chr'] + ':' + df['phe_to_start'].astype(str) + '-' + df['phe_to'].astype(str) + ',' + df['slope'].astype(str)
    df = df[['phe_chr', 'phe_from', 'phe_from_end', 'combined', "log_pval"]]
    df.to_csv(f"{base_dir}/QTL_analysis/ins_output/loop_CD8_{wanted_snp}.longrange", sep='\t', index=False, header = False)
    df = loop_nominal_CD4[loop_nominal_CD4["var_id"] == wanted_snp].copy()
    df['combined'] = df['phe_chr'] + ':' + df['phe_to_start'].astype(str) + '-' + df['phe_to'].astype(str) + ',' + df['slope'].astype(str)
    df = df[['phe_chr', 'phe_from', 'phe_from_end', 'combined', "log_pval"]]
    df.to_csv(f"{base_dir}/QTL_analysis/ins_output/loop_CD4_{wanted_snp}.longrange", sep='\t', index=False, header = False)